Input Properties Processing

In [ ]:
#importing all the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from pandas import DataFrame
import sklearn
from pandas import DataFrame
from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import random
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
import shap

In [ ]:
#Dataset

dataset=pd.read_csv(r"HEI New.csv")

In [ ]:
#Normalising datapoints

df=dataset[["Fe","Mn","Ni","Ti","Mo","N","S","C","Al","Si","Nb","Cu","V","Cr","P","Grain size", "HEI","current density/Pressure","Time","Strain rate",	"Ion Radius_Avg", "Bulk Modulus_Avg",		"Shear Modulus_Avg",	"Effective nuclear charge_Avg",		"Valence electron Concentration_Avg","Allen Electronegativity_Avg","Cohesive energy_Avg"]]
df1=dataset[["Fe","Mn","Ni","Ti","Mo","N","S","C","Al","Si","Nb","Cu","V","Cr","P","Grain size", "HEI","current density/Pressure",	"Time",	"Strain rate",	"YS",	"UTS",	"UTS ductility","Fracture ductility","Ion Radius_Avg", "Bulk Modulus_Avg","Shear Modulus_Avg","Effective nuclear charge_Avg","Valence electron Concentration_Avg","Allen Electronegativity_Avg","Cohesive energy_Avg"]]

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(df)
df_normalized = pd.DataFrame(x_scaled)

#Saving normalised datapoints into 
df_normalized.to_csv(r"Regression_NewNormalisedElectrochargingPropertyData_InputProperties.csv", header = ["Fe","Mn","Ni","Ti","Mo","N","S","C","Al","Si","Nb","Cu","V","Cr","P","Grain size","HEI", "current density/Pressure",	"Time",	"Strain rate","Ion Radius_Avg", "Bulk Modulus_Avg","Shear Modulus_Avg","Effective nuclear charge_Avg","Valence electron Concentration_Avg","Allen Electronegativity_Avg","Cohesive energy_Avg"])

In [ ]:
Input = pd.read_csv(r"Regression_NewNormalisedElectrochargingPropertyData_InputProperties.csv", header = 0)
x = Input.loc[:, ["Fe","Mn","Ni","Ti","Mo","N","S","C","Al","Si","Nb","Cu","V","Cr","P","Grain size", "current density/Pressure",	"Time",	"Strain rate","Ion Radius_Avg", "Bulk Modulus_Avg","Shear Modulus_Avg","Effective nuclear charge_Avg","Valence electron Concentration_Avg","Allen Electronegativity_Avg","Cohesive energy_Avg"]]
y= df1.loc[:, ["HEI"]]

In [ ]:
#Plotting heatmap

plt.figure(figsize=(120, 100))  # Set the size of the figure
sns.heatmap(x.corr(), cmap="viridis", annot=True, annot_kws={"fontsize": 64})  # Set the fontsize parameter
plt.show()

In [ ]:
#Feature Importance Score calculation using Gradient Boosting 

gb_reg = GradientBoostingRegressor(loss= 'squared_error', learning_rate=0.1, n_estimators=100, max_depth=3)
gb_reg.fit(x,y.values.ravel())
feature_importances= gb_reg.feature_importances_

for feature_name, importance_score in zip(x.columns, feature_importances):
    print(f"{feature_name}: {importance_score}")

In [ ]:
#Ploting Feature Imposrtance Score

plt.figure(figsize=(10, 6))
plt.barh(x.columns, feature_importances)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title('Feature Importances')
plt.gca().invert_yaxis()  # Invert y-axis to display most important feature on top
plt.show()

Model Training for UTS Prediction

In [ ]:
Input = pd.read_csv(r"Regression_NewNormalisedElectrochargingPropertyData_InputProperties.csv", header = 0)
x = Input.loc[:, ["Fe","Mn","Ni","Ti","Mo","N","S","C","Al","Si","Nb","Cu","V","Cr","P","Grain size", "current density/Pressure",	"Time",	"Strain rate","Ion Radius_Avg", "Bulk Modulus_Avg","Shear Modulus_Avg","Effective nuclear charge_Avg","Valence electron Concentration_Avg","Allen Electronegativity_Avg","Cohesive energy_Avg"]]
y1= df1.loc[:, ["UTS"]]

In [ ]:
#Splitting into test and train datapoints
x_train, x_test, y_train, y_test = train_test_split(x, y1, random_state =42 ,test_size = 0.2)

# Define the pipelines
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()), ('LR', LinearRegression())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()), ('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()), ('GBM', GradientBoostingRegressor())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()), ('LASSO', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()), ('EN', ElasticNet())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()), ('CART', DecisionTreeRegressor())])))

results = []
names = []
r2_scores = []

# Evaluate each model using cross-validation
for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=42, shuffle=True)
    cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring='r2')
    results.append(cv_results)
    names.append(name)
    r2_mean = cv_results.mean()
    r2_scores.append(r2_mean)
    print(f"{name}: R2 score mean: {r2_mean}")

# Find the best performing model
best_model_idx = np.argmax(r2_scores)
best_model_name = names[best_model_idx]
best_model_r2 = r2_scores[best_model_idx]
print(f"\nBest performing model: {best_model_name} with R2 score mean: {best_model_r2}")

In [ ]:
# reshape y1_train and y1_test into 1D arrays 
y1_train = np.ravel(y_train)
y1_test = np.ravel(y_test)

# create the pipeline object
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf_regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# fit the pipeline to the training data
rf_pipeline.fit(x_train, y1_train)

# Extract feature importances
feature_importances = rf_pipeline.named_steps['rf_regressor'].feature_importances_

# Extract the coefficients (feature importances) and intercept
coefficients = feature_importances
# Create a dummy input array with zeros for calculating the intercept
dummy_input = np.zeros((1, len(x_train.columns))) if isinstance(x_train, pd.DataFrame) else np.zeros((1, len(x_train[0])))
intercept = rf_pipeline.named_steps['rf_regressor'].predict(dummy_input)[0]

# Print the equation
print("Equation:")
equation = " + ".join([f"{coeff:.4f} * x{i+1}" for i, coeff in enumerate(coefficients)])
print(f"y = {equation} + {intercept:.4f}")

# make predictions on the test set
y1_pred = rf_pipeline.predict(x_test)

# calculate the R2 score
r2 = r2_score(y1_test, y1_pred)

# print the R2 score
print(f"R2 score: {r2:.3f}")

In [ ]:
#Calculate Scoring metrics 

y1_pred_test_EN = rf_pipeline.predict(x_test)
y1_pred_train_EN = rf_pipeline.predict(x_train)

r2_train = metrics.r2_score(y_train, y1_pred_train_EN)
adj_r2_train = 1 - (1-r2_train)*(len(x_train)-1)/(len(x_train)-df.shape[1]-1)
r2_test = metrics.r2_score(y_test, y1_pred_test_EN)
adj_r2_test = 1 - (1-r2_test)*(len(x_train)-1)/(len(x_train)-df.shape[1]-1)


print('For Train: ')
print('R squared value:', metrics.r2_score(y_train, y1_pred_train_EN))
print('Adjusted R2Score: ', adj_r2_train)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_train, y1_pred_train_EN))
print('Mean Squared Error:', metrics.mean_squared_error(y_train, y1_pred_train_EN))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y1_pred_train_EN)))
#print('Mean Relative Error:', mre_train)
print('For Test: ')
print('R squared value:', metrics.r2_score(y_test, y1_pred_test_EN))
print('Adjusted R2Score: ', adj_r2_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y1_pred_test_EN))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y1_pred_test_EN))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y1_pred_test_EN)))

In [ ]:
print("y_test:", y1_test)
print("y_pred:", y1_pred)

# Save y_test and y_pred to a CSV file
with open('utsdata.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['y_test', 'y_pred'])
    for yt, yp in zip(y1_test, y1_pred):
        writer.writerow([yt, yp])

# Plotting Reported Vs Predicted
plt.scatter(y1_pred, y1_test)
plt.plot(y1_pred, y1_test)
plt.xlabel('Predicted UTS')
plt.ylabel('Actual UTS')
plt.title('UTS')
plt.show()

In [ ]:
# Initialize the SHAP explainer with the trained Random Forest Regressor model
explainer = shap.Explainer(rf_pipeline.named_steps['rf_regressor'], x_train)

# Calculate SHAP values for all samples in the test set
shap_values = explainer.shap_values(x_test)

# Calculate mean SHAP values for each property
mean_shap_values = shap_values.mean(axis=0)

# Print mean SHAP values for each property
for property_name, mean_shap_value in zip(x_train.columns, mean_shap_values):
    print(f"Mean SHAP Value for {property_name}: {mean_shap_value}")

# Summarize the impact of all features (bar plot)
shap.summary_plot(shap_values, x_test, plot_type='bar', feature_names=x_train.columns)

# Summarize the impact of all features
shap.summary_plot(shap_values, x_test)

In [ ]:
#Training GBR model
GBR = GradientBoostingRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }
grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1)
grid_GBR.fit(x_train, y_train)

print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.04, max_depth=6, subsample=0.2)
gbr.fit(x_train, y_train)

pred_y1 = gbr.predict(x_test)

R2 = r2_score(pred_y1, y_test)
print(f"R2: {R2:.3f}")

In [ ]:
#Elastic Net Training

# Define the pipeline with polynomial features and elastic net
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=3)),
    ('scaler', StandardScaler()),
    ('elasticnet', ElasticNet(max_iter=1000000))
])

# Define the hyperparameter grid to search over
param_grid = {
    'elasticnet__alpha': [0.01, 0.05, 0.08, 0.09, 0.1, 0.5, 1.0, 10.0, 100.0],
    'elasticnet__l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
}

# Define the GridSearchCV object to run the search
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the model on the training data
grid_search.fit(x_train, y_train)

# Get the best hyperparameters and score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Get the predictions on the test data
y_pred = grid_search.predict(x_test)

# Get the R2 score on the test data
r2_score = r2_score(y_test, y_pred)
print("R2 score on test data: ", r2_score)

In [ ]:
print('Elastic Net Regressor:- ')
print("Best Parameters:", grid_search.best_params_)

print("")

print("Cross-validation score:{}".format(grid_search.best_score_))
print("")

y_pred_test_EN = grid_search.predict(x_test)
y_pred_train_EN = grid_search.predict(x_train)

In [ ]:
r2_train = metrics.r2_score(y_train, y_pred_train_EN)
adj_r2_train = 1 - (1-r2_train)*(len(x_train)-1)/(len(x_train)-df.shape[1]-1)
r2_test = metrics.r2_score(y_test, y_pred_test_EN)
adj_r2_test = 1 - (1-r2_test)*(len(x_train)-1)/(len(x_train)-df.shape[1]-1)


print('For Train: ')
print('R squared value:', metrics.r2_score(y_train, y_pred_train_EN))
print('Adjusted R2Score: ', adj_r2_train)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred_train_EN))
print('Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred_train_EN))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train_EN)))
#print('Mean Relative Error:', mre_train)
print('For Test: ')
print('R squared value:', metrics.r2_score(y_test, y_pred_test_EN))
print('Adjusted R2Score: ', adj_r2_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_test_EN))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_test_EN))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test_EN)))
#print('Mean Relative Error:', mre_test)

Model Training for Ductility Prediction

In [ ]:
Input = pd.read_csv(r"Regression_NewNormalisedElectrochargingPropertyData_InputProperties.csv", header = 0)
x = Input.loc[:, ["Fe","Mn","Ni","Ti","Mo","N","S","C","Al","Si","Nb","Cu","V","Cr","P","Grain size", "current density/Pressure",	"Time",	"Strain rate","Ion Radius_Avg", "Bulk Modulus_Avg","Shear Modulus_Avg","Effective nuclear charge_Avg","Valence electron Concentration_Avg","Allen Electronegativity_Avg","Cohesive energy_Avg"]]
y2= df1.loc[:, ["UTS Ductility"]]

In [ ]:
x_train, x_test, y2_train, y2_test = train_test_split(x, y2, random_state =42 ,test_size = 0.2)

# Define the pipelines
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()), ('LR', LinearRegression())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()), ('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()), ('GBM', GradientBoostingRegressor())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()), ('LASSO', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()), ('EN', ElasticNet())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()), ('CART', DecisionTreeRegressor())])))

results = []
names = []
r2_scores = []

# Evaluate each model using cross-validation
for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=42, shuffle=True)
    cv_results = cross_val_score(model, x_train, y2_train, cv=kfold, scoring='r2')
    results.append(cv_results)
    names.append(name)
    r2_mean = cv_results.mean()
    r2_scores.append(r2_mean)
    print(f"{name}: R2 score mean: {r2_mean}")

# Find the best performing model
best_model_idx = np.argmax(r2_scores)
best_model_name = names[best_model_idx]
best_model_r2 = r2_scores[best_model_idx]
print(f"\nBest performing model: {best_model_name} with R2 score mean: {best_model_r2}")


In [ ]:
y2_train = np.ravel(y2_train)
y2_test = np.ravel(y2_test)

# create the pipeline object
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf_regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# fit the pipeline to the training data
rf_pipeline.fit(x_train, y2_train)

# Extract feature importances
feature_importances = rf_pipeline.named_steps['rf_regressor'].feature_importances_

# Extract the coefficients (feature importances) and intercept
coefficients = feature_importances
# Create a dummy input array with zeros for calculating the intercept
dummy_input = np.zeros((1, len(x_train.columns))) if isinstance(x_train, pd.DataFrame) else np.zeros((1, len(x_train[0])))
intercept = rf_pipeline.named_steps['rf_regressor'].predict(dummy_input)[0]

# Print the equation
print("Equation:")
equation = " + ".join([f"{coeff:.4f} * x{i+1}" for i, coeff in enumerate(coefficients)])
print(f"y = {equation} + {intercept:.4f}")

# make predictions on the test set
y2_pred = rf_pipeline.predict(x_test)

# calculate the R2 score
r2 = r2_score(y2_test, y2_pred)

# print the R2 score
print(f"R2 score: {r2:.3f}")


# make predictions on the test set
y2_pred = rf_pipeline.predict(x_test)

# calculate the MAE value
mae = mean_absolute_error(y2_test, y2_pred)

# print the MAE value
print(f"MAE value: {mae:.3f}")

In [ ]:
y2_pred_test_EN = rf_pipeline.predict(x_test)
y2_pred_train_EN = rf_pipeline.predict(x_train)

r2_train = metrics.r2_score(y2_train, y2_pred_train_EN)
adj_r2_train = 1 - (1-r2_train)*(len(x_train)-1)/(len(x_train)-df.shape[1]-1)
r2_test = metrics.r2_score(y2_test, y2_pred_test_EN)
adj_r2_test = 1 - (1-r2_test)*(len(x_train)-1)/(len(x_train)-df.shape[1]-1)


print('For Train: ')
print('R squared value:', metrics.r2_score(y2_train, y2_pred_train_EN))
print('Adjusted R2Score: ', adj_r2_train)
print('Mean Absolute Error:', metrics.mean_absolute_error(y2_train, y2_pred_train_EN))
print('Mean Squared Error:', metrics.mean_squared_error(y2_train, y2_pred_train_EN))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_train, y2_pred_train_EN)))
#print('Mean Relative Error:', mre_train)
print('For Test: ')
print('R squared value:', metrics.r2_score(y2_test, y2_pred_test_EN))
print('Adjusted R2Score: ', adj_r2_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred_test_EN))
print('Mean Squared Error:', metrics.mean_squared_error(y2_test, y2_pred_test_EN))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred_test_EN)))
#print('Mean Relative Error:', mre_test)

In [ ]:
# Print y_test and y_pred
print("y2_test:", y2_test)
print("y2_pred:", y2_pred)

# Save y_test and y_pred to a CSV file
with open('utsductilitydata.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['y2_test', 'y2_pred'])
    for yt, yp in zip(y2_test, y2_pred):
        writer.writerow([yt, yp])

# Plot
plt.scatter(y2_pred, y2_test)
plt.plot(y2_pred, y2_test)
plt.xlabel('Predicted UTS')
plt.ylabel('Actual UTS')
plt.title('UTS')
plt.show()

In [ ]:
# Initialize the SHAP explainer with the trained Random Forest Regressor model
explainer = shap.Explainer(rf_pipeline.named_steps['rf_regressor'], x_train)

# Calculate SHAP values for all samples in the test set
shap_values = explainer.shap_values(x_test)

# Calculate mean SHAP values for each property
mean_shap_values = shap_values.mean(axis=0)

# Print mean SHAP values for each property
for property_name, mean_shap_value in zip(x_train.columns, mean_shap_values):
    print(f"Mean SHAP Value for {property_name}: {mean_shap_value}")

# Summarize the impact of all features (bar plot)
shap.summary_plot(shap_values, x_test, plot_type='bar', feature_names=x_train.columns)

# Summarize the impact of all features
shap.summary_plot(shap_values, x_test)

In [ ]:
#Gradient Boosting Model Training
GBR = GradientBoostingRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }
grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1)
grid_GBR.fit(x_train, y2_train)

print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.04, max_depth=6, subsample=0.2)
gbr.fit(x_train, y2_train)

pred_y1 = gbr.predict(x_test)

R2 = r2_score(y2_pred, y2_test)
print(f"R2: {R2:.3f}")

In [ ]:
# Define the pipeline with polynomial features and elastic net
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=3)),
    ('scaler', StandardScaler()),
    ('elasticnet', ElasticNet(max_iter=1000000))
])

# Define the hyperparameter grid to search over
param_grid = {
    'elasticnet__alpha': [0.01, 0.05, 0.08, 0.09, 0.1, 0.5, 1.0, 10.0, 100.0],
    'elasticnet__l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
}

# Define the GridSearchCV object to run the search
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y2, test_size=0.2, random_state=42)

# Fit the model on the training data
grid_search.fit(x_train, y_train)

# Get the best hyperparameters and score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Get the predictions on the test data
y_pred = grid_search.predict(x_test)

# Get the R2 score on the test data
r2 = r2_score(y_test, y_pred)
print("R2 score on test data: ", r2)

# Initialize the SHAP explainer with the trained Elastic Net model
explainer = shap.Explainer(grid_search.best_estimator_.named_steps['elasticnet'], x_train)

# Calculate SHAP values for all samples in the test set
shap_values = explainer.shap_values(x_test)

# Summarize the impact of all features
shap.summary_plot(shap_values, x_test, plot_type='bar', feature_names=x_train.columns)

# Display the SHAP values for individual predictions (you can choose specific instances)
shap.summary_plot(shap_values, x_test)


In [ ]:
print('Elastic Net Regressor:- ')
print("Best Parameters:", grid_search.best_params_)

print("")

print("Cross-validation score:{}".format(grid_search.best_score_))
print("")

y2_pred_test_EN = grid_search.predict(x_test)
y2_pred_train_EN = grid_search.predict(x_train)

In [ ]:
r2_train = metrics.r2_score(y_train, y2_pred_train_EN)
adj_r2_train = 1 - (1-r2_train)*(len(x_train)-1)/(len(x_train)-df.shape[1]-1)
r2_test = metrics.r2_score(y_test, y2_pred_test_EN)
adj_r2_test = 1 - (1-r2_test)*(len(x_train)-1)/(len(x_train)-df.shape[1]-1)


print('For Train: ')
print('R squared value:', metrics.r2_score(y_train, y2_pred_train_EN))
print('Adjusted R2Score: ', adj_r2_train)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_train, y2_pred_train_EN))
print('Mean Squared Error:', metrics.mean_squared_error(y_train, y2_pred_train_EN))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y2_pred_train_EN)))
#print('Mean Relative Error:', mre_train)
print('For Test: ')
print('R squared value:', metrics.r2_score(y_test, y2_pred_test_EN))
print('Adjusted R2Score: ', adj_r2_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y2_pred_test_EN))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y2_pred_test_EN))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y2_pred_test_EN)))
#print('Mean Relative Error:', mre_test)